In [8]:
from elasticsearch import Elasticsearch, helpers
import geopandas as gpd
import pandas as pd

# connect to Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

# load sample data
sample = pd.read_csv('./data/ukpostcodes.csv',
                     usecols=['postcode', 'latitude', 'longitude'])

man_postcodes = sample[sample.postcode.str.startswith('M')]

In [9]:
man_postcodes.head()

,postcode,latitude,longitude
723093,ML9 3PN,55.701404,-3.951959
723094,ML9 3PP,55.697406,-3.944010
723095,ML9 3PQ,55.703840,-3.993938
723096,ML9 3PR,55.689201,-3.953610
723097,ML9 3PS,55.682686,-3.961749
